# Tutorial Session #2, Simple Imaging

Huib Jan van Langevelde / Michiel Brentjens

In this session we will try to appreciate the basic concepts of synthesis imaging. The data set for this session was taken with the VLA (in the distant past) and allows us to image the centre of our Milky Way, the radio source Sagittarius A.

The data are found in the current directory:

In [1]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [2]:
! ls *.uvf

p1conc.uvf  sgrac.uvf


You will be using CASA which starts by typing ‘casa570’ (to get the most recent stable for 2018-09-21) at the Unix prompt. Lots of background information for casa can be found at http://casa.nrao.edu/.

It is helpful to collect all the commands you need into a separate textfile to copy-and-paste from when needed. You can use the "+" under the Jupyter menu to start a new "Launcher" to create a new text file that you can rename `sgrastar-reduction.py` or any other name you deem useful.

Alternatively, use any text editor you like.

If you ever need to re-run your entire script, you can execute `execfile('SCRIPT_NAME.py')`, which loads the commands in the file as if you were typing them in, with the only difference that if you want to use "`inp`" and "`go`" in your script, you'll need to put their arguments in parentheses.

## Import

The data are stored in `uvfits` format, and first need to be imported using the `importuvfits` task. At the casa prompt type:

    inp importuvfits
    fitsfile = 'p1conc.uvf'
    vis = 'p1conc.MS'

Verify the parameters by issuing:

    inp
    
once more.

Once happy with the result:

    go importuvfits
    
or simply 

    go

Now have a look at the Casa log window. Identify the line that gives the actual Python command.  You can copy-and-paste that line into your data reduction text file to log it for the future.

The visibility database (Measurement Set, or MS) is then available as a Casa MS named 'p1conc.MS'. Have a look at its structure:

In [3]:
! ls -l p1conc.MS

total 10888
drwxr-xr-x 1 brentjens brentjens      74 Sep 27  2018 ANTENNA
drwxr-xr-x 1 brentjens brentjens      74 Sep 27  2018 DATA_DESCRIPTION
drwxr-xr-x 1 brentjens brentjens      92 Sep 27  2018 FEED
drwxr-xr-x 1 brentjens brentjens      92 Sep 27  2018 FIELD
drwxr-xr-x 1 brentjens brentjens      74 Sep 27  2018 FLAG_CMD
drwxr-xr-x 1 brentjens brentjens      74 Sep 27  2018 HISTORY
drwxr-xr-x 1 brentjens brentjens      74 Sep 27  2018 OBSERVATION
drwxr-xr-x 1 brentjens brentjens     108 Sep 27  2018 POINTING
drwxr-xr-x 1 brentjens brentjens      92 Sep 27  2018 POLARIZATION
drwxr-xr-x 1 brentjens brentjens      74 Sep 27  2018 PROCESSOR
drwxr-xr-x 1 brentjens brentjens      92 Sep 27  2018 SOURCE
drwxr-xr-x 1 brentjens brentjens      92 Sep 27  2018 SPECTRAL_WINDOW
drwxr-xr-x 1 brentjens brentjens      74 Sep 27  2018 STATE
-rw-r--r-- 1 brentjens brentjens   10526 Sep 27  2018 table.dat
-rw-r--r-- 1 brentjens brentjens   63050 Sep 27  2018 table.f0
-rw-r--r-- 1 brentjens brentjens 

The `table.*` files contain the data in the main table columns. The sub directories (`ANTENNA`..`STATE`) are sub-tables containing "metadata": data describing the visibility data sets.

At the Casa prompt, you may issue the command `%history` if you want to obtain a log of all the commands you issued in a certain session. This is extremely helpful when you want to write or clean up your data reduction script.

## Inspection


Inspect the data with `vishead` and/or `listobs`. You should discover there were 2 sources observed with many VLA antennas. Take note of the observing date, observing frequencies. You'll find them in the log window. There are two correlator polarisation products.

You will also need to know how the antennas are distributed. This can be done with ‘plotants'. Estimate what the longest baseline is for these observations. Also make a note which antennas are near the centre of the array. You will need this later.

### ANSWER

Antennas 17, 18, 19, 23, 25, 29 are very close to the array centre.

The longest baseline is from antenna 15 to 12, and is (in x-y) coordinates in the plotants view approximately `sqrt(3000**2+3000**2 + 3500**2 + 5000**2)` = 7500 m


Using the full 3D coordinates from the log window (`listobs` output), one ends up at:

    2018-09-25 20:02:32 INFO listobs	  0    1     ANT1      25.0 m   -107.35.14.0  +33.51.51.3       2871.6880    -4030.1469     1667.2848 -1599545.338843 -5046307.474025  3552459.163647
    2018-09-25 20:02:32 INFO listobs	  1    2     ANT2      25.0 m   -107.38.04.6  +33.54.54.1      -1503.8759     1618.3450     1991.6956 -1602845.854958 -5042236.946372  3557330.286199
    2018-09-25 20:02:32 INFO listobs	  3    4     ANT4      25.0 m   -107.36.16.0  +33.53.03.5       1283.6202    -1798.4674      743.9645 -1600452.017662 -5043910.527305  3553797.760523
    2018-09-25 20:02:32 INFO listobs	  4    5     ANT5      25.0 m   -107.37.11.7  +33.54.25.9       -145.3281      745.1668    -3096.4429 -1600420.154532 -5039088.098497  3553766.991631
    2018-09-25 20:02:32 INFO listobs	  5    6     ANT6      25.0 m   -107.38.38.8  +33.55.24.6      -2380.9414     2561.3799     3153.5062 -1603814.455089 -5042388.198216  3558761.474117
    2018-09-25 20:02:32 INFO listobs	  6    7     ANT7      25.0 m   -107.37.17.4  +33.54.50.1       -291.4217     1494.9906    -6208.9075 -1599650.804822 -5036183.634280  3552652.385459
    2018-09-25 20:02:32 INFO listobs	  7    8     ANT8      25.0 m   -107.37.34.3  +33.54.27.0       -725.9313      779.2972      959.6099 -1601986.762560 -5042102.520075  3556057.987851
    2018-09-25 20:02:32 INFO listobs	  8    9     ANT9      25.0 m   -107.36.55.6  +33.53.49.7        267.0320     -372.9458      154.4717 -1601032.605603 -5042378.818787  3554652.465530
    2018-09-25 20:02:32 INFO listobs	  9    10    ANT10     25.0 m   -107.35.36.5  +33.52.18.3       2295.6489    -3196.0442     1327.1352 -1599868.918074 -5045420.854962  3552962.342408
    2018-09-25 20:02:32 INFO listobs	  10   11    ANT11     25.0 m   -107.36.45.1  +33.53.37.5        535.2651     -747.9085      309.7189 -1600879.197455 -5042782.129460  3554427.792035
    2018-09-25 20:02:32 INFO listobs	  11   12    ANT12     25.0 m   -107.34.49.2  +33.51.21.9       3507.4173    -4937.9648     2040.3956 -1599185.299424 -5047277.343275  3551912.978033
    2018-09-25 20:02:32 INFO listobs	  12   13    ANT13     25.0 m   -107.38.19.9  +33.55.07.7      -1894.1500     2036.5588     2507.5101 -1603276.806284 -5042304.140632  3557965.281771
    2018-09-25 20:02:32 INFO listobs	  13   14    ANT14     25.0 m   -107.37.15.3  +33.54.41.3       -238.5054     1220.8292    -5072.7552 -1599932.118554 -5037243.931994  3553058.955695
    2018-09-25 20:02:32 INFO listobs	  14   15    ANT15     25.0 m   -107.38.59.6  +33.55.43.2      -2913.7115     3134.8754     3860.1543 -1604402.946662 -5042480.216028  3559631.995143
    2018-09-25 20:02:32 INFO listobs	  15   16    ANT16     25.0 m   -107.37.13.3  +33.54.33.2       -187.6742      972.1789    -4033.7622 -1600186.696455 -5038213.240238  3553432.394267
    2018-09-25 20:02:32 INFO listobs	  16   17    ANT17     25.0 m   -107.37.08.6  +33.54.04.1        -67.7540       73.7696       90.3569 -1601260.187002 -5041989.302069  3554987.496730
    2018-09-25 20:02:32 INFO listobs	  17   18    ANT18     25.0 m   -107.37.06.3  +33.54.02.9         -6.5177       34.6448     -143.1728 -1601149.760088 -5041843.897330  3554824.769766
    2018-09-25 20:02:32 INFO listobs	  18   19    ANT19     25.0 m   -107.37.02.8  +33.53.58.1         81.6521     -113.0590       47.0343 -1601138.443538 -5042099.569299  3554808.262413
    2018-09-25 20:02:32 INFO listobs	  19   20    ANT20     25.0 m   -107.36.31.9  +33.53.22.1        875.1317    -1226.4221      507.1807 -1600685.529835 -5043295.544192  3554140.664701
    2018-09-25 20:02:32 INFO listobs	  20   21    ANT21     25.0 m   -107.37.14.6  +33.54.09.4       -220.6549      237.4695      292.1143 -1601428.966480 -5042015.593129  3555235.905453
    2018-09-25 20:02:32 INFO listobs	  21   22    ANT22     25.0 m   -107.37.09.0  +33.54.13.7        -76.8668      368.1548    -1544.8112 -1600808.375627 -5040536.557260  3554319.739524
    2018-09-25 20:02:32 INFO listobs	  22   23    ANT23     25.0 m   -107.37.06.9  +33.54.05.4        -23.7149      111.8489     -470.3173 -1601070.930421 -5041538.859187  3554706.377588
    2018-09-25 20:02:32 INFO listobs	  23   24    ANT24     25.0 m   -107.37.10.1  +33.54.19.4       -105.8352      545.2370    -2264.5851 -1600625.260593 -5039864.310582  3554065.169822
    2018-09-25 20:02:32 INFO listobs	  24   25    ANT25     25.0 m   -107.37.07.9  +33.54.09.0        -47.5371      224.2594     -943.0361 -1600955.898355 -5041097.949012  3554535.996895
    2018-09-25 20:02:32 INFO listobs	  25   26    ANT26     25.0 m   -107.37.47.5  +33.54.38.8      -1063.6648     1143.4287     1407.4932 -1602359.701273 -5042160.850217  3556610.109166
    2018-09-25 20:02:32 INFO listobs	  26   27    ANT27     25.0 m   -107.37.23.3  +33.54.17.1       -443.2384      475.6471      585.7548 -1601674.670105 -5042053.856010  3555597.393098
    2018-09-25 20:02:32 INFO listobs	  27   28    ANT28     25.0 m   -107.35.57.5  +33.52.42.2       1757.4964    -2456.9648     1017.8111 -1600179.939253 -5044620.538537  3553403.667235


Result:`sqrt(2913**2 + 3135**2 + 3860**2+ 3507**2 + 4937**2 + 2040**2) = 8685 m`

## Calibrator, data inspection

We will now concentrate First on the calibrator. You will need to set `field=‘0’` when running tasks for that. Plot the amplitude of this source in time (`plotms`), this shows when the data was observed. Also note that there is significant signal, constant in time. Of course it is not calibrated yet.

In the same tool you can also plot the uv-coverage by plotting U versus V (use the "Axes" tab on the left). Note that this is not a 12h full observation, but you can make reasonable images with the VLA anyway. An `amp`litude versus `UVdist` plot (also using `plotms`) shows the source is detectable on all baselines. But plotting the (`UVdist`, `phase`) plot shows the array is not focused at all. 

Explain why.

### ANSWER

The calibrator should be a point source at the phase centre of the array. If the array is well focused, a point source at the field's phase centre should have 0 phase for all baselines. This is *not* what we see: phases are uniformly distributed between $-180^\circ$ and $+180^\circ$. 

Try to see the phase on separate baselines in time by selecting baselines with a central antenna (this requires mastering the `‘plotms’` interfaces). It is illustrative to image the source from the un-calibrated data. So run `tclean` without doing any cleaning (`niter=0`). Most other parameters can be left at default, except you must select a sensible
image name and you probably want to switch `‘interacive=True’`. And of course you must worry about the resolution.

You should be able to estimate the resolution from the uv-coverage and observing wavelength. Typically you need 3 pixels across the beam. Start with a reasonably sized image of 512 pixels across. The flux is scattered around and no image is obtained. Inspect the other products like the pointspread function(PSF) image with `‘viewer’`.

In [4]:
! ls -lrtd  *.*

-rw-r--r-- 1 brentjens brentjens 6033600 Sep 21  2018 p1conc.uvf
-rw-r--r-- 1 brentjens brentjens 3191040 Sep 21  2018 sgrac.uvf
-rw-r--r-- 1 brentjens brentjens      88 Sep 25  2018 casa-20180925-140145.log
-rw-r--r-- 1 brentjens brentjens  969604 Sep 27  2018 casa-20180925-140626.log
-rw-r--r-- 1 brentjens brentjens   79181 Sep 27  2018 casa-20180927-050254.log
-rw-r--r-- 1 brentjens brentjens   67860 Sep 27  2018 casa-20180927-083221.log
-rw-r--r-- 1 brentjens brentjens   65250 Sep 27  2018 casa-20180927-084102.log
-rw-r--r-- 1 brentjens brentjens   64301 Sep 27  2018 casa-20180927-084455.log
-rw-r--r-- 1 brentjens brentjens   80864 Sep 27  2018 casa-20180927-084840.log
-rw-r--r-- 1 brentjens brentjens      89 Sep 27  2018 casa-20180927-085821.log
-rw-r--r-- 1 brentjens brentjens   18335 Sep 27  2018 casa-20180927-090354.log
-rw-r--r-- 1 brentjens brentjens   60480 Sep 27  2018 casa-20180927-090536.log
-rw-r--r-- 1 brentjens brentjens   55853 Sep 27  2018 casa-20180927-091123.log
-r

## Calibrate

Now it is time to start calibrating. We will work on the assumption that 1748-2530 is a point source with 1.17 Jy of flux. Normally this number is derived from a flux calibrator, but for now trust us that we have done this correctly for you. Setup `gaincal` to write a calibration table (you have to define `caltable`) with the proper `‘smodel’`, a 5 min interval is sufficient at 18 cm to counter the ionospheric fluctuations over the telescopes, and set one of the center antennas as the reference. 

Which `calmode` should we use?

### ANSWER

`calmode` should be `'ap'`. We want to focus the array by aligning all antenna phases, as well as determine the gain amplitude to establish an absolute flux density scale. 

Be sure to only select data on the calibrator. The resulting solutions can be inspected with `‘plotcal’`. Make sure you see that the phase solutions ‘hang together’ by plotting the phase solutions in time. Why are all the solutions for your reference antenna 0? Amplitudes should be similar for all antennas. It is useful to plot the SNR against time too.

Run `‘applycal’` with mostly default values applying your gain table to all the sources in the data. Now repeat the plots (`plotms`) you did before with this calibration applied. For this you must select the `‘datacolumn=corrected’` in the Axes pane. The original uncalibrated data is still there in the data column. Phase versus time should show the array is now in focus. Amplitude versus UV-distance shows the data is consistent with a 1.17 Jy point source. And phase versus UV-distance shows it is located at the centre.

You are ready to make a map, run `tclean` again with a few iterations (`niter=200`). Make sure you draw a "clean box" around the central source using your right mouse button, and click inside that region with your right mouse button before pressing the "semi-circle-arrow/continue" button at the top of the interface. Inspect the result in `‘viewer’`.

In `viewer` adjust the data display to stretch the brightness scale until you can see the noise in the map. Is the central source alone? How much fainter is the next brightest "believable" source?

### ANSWER

No, there are at least two others. The brightest of those appears to be slightly extended and is of the order of 11 mJy bright: a factor $\sim100$ fainter than 1748-2530.

## Target in B array

The dataset (that you just calibrated) also contains data on the Galactic centre, Sgr A. You can inspect the data in `‘plotms’` by selecting the `field=‘1’`. You can switch between the `column=data` and `column=corrected` to see the effect of the calibration. Note that the plot of (amp, uvdist) looks quite different for this source. How much flux is there in this field?

### ANSWER

The shortest baselines see all flux, regardless of scale, except the total offset. The highest baseline flux we see is about 144 Jy, implying that there is *at least* 144 Jy of flux in the field. 

Remember this number, becaus we should later on compare our total model flux to it to see if we have captured all the flux in the field.

Now clean this data, note that you will a bit larger map this time (`imsize = 1024`) and take a larger number of clean components, e.g. `niter=10000`. Imaging and cleaning this will take a few measures. 


## The "Old" way
You will need to master how to set boxes to constrain the process. You may want to reduce the gain to proceed more carefully. This dataset is particularly sensitive to different weighting, try `‘weighting=uniform’` or even robust weighting (`weighting=‘briggs’`). You will need to update the clean boxes along the way.

During the deconvolution, watch the log window for information on total flux captured in the model. How dows it compare to the total flux derived from the shortest baselines?

Notably, this data suffers from incomplete data on the short baselines. This can be beautifully illustrated by restricting yourself to the long baselines. You can use (`uvrange=‘>10klambda’`) to make an image that filters all the extended emission. The point source you discover this way is Sgr A*, the radio emission associated with the black hole in the Galactic centre!

Make sure you measure the noise statistics in this image and compare it to the other images you have created on target and calibrator.


## The "New" way

You will have noticed that there is a lot of extended emission in this field. In addition, setting "clean" masks or boxes after every few hundred iterations becomes cumbersome quickly.

`tclean` has a few interesting features to make the entire experience a lot more convenient.

The standard "Hogbom" clean attempts to model each source as a bed-of-nails. This is fine if the field consists mainly of point sources, but not in this case. However, at a sufficiently low resolution, every source is a point source. Enter `multiscale` clean. When `deconvolver='multiscale'`, `tclean` concurrently deconvolves images at various amounts of smoothing. A clean component found at a scale of 16 pixels, is rendered on the model image as a Gaussian blob of width 16. You can specify the scales to find components for using the `scales` parameter. Usually, `scales` of powers of two (with 0 representing an actual point), work well: `scales=[0,1,2,4,8,16,32,64,128]`.

When drawing your clean masks in the previous section, you have probably looked at your residual image, and drawn lines around the brightest patches. This is of course a perfect job for a computer. When you set `usemask='auto-multithresh'`, `tclean` will automatically determine a clean mask per deconvolver scale!

Compare the final images from the "old" and the "new" way, as well as their model images.


## Optional: Combine with C array

There is an even  better way to improve the overall image, namely by acquiring more data on shorter baselines. The VLA can provide this. The data you had first was ‘B array’; we also have some C array data in which the antennas are moved closer together, providing shorter baselines. And it is already calibrated.

Read data from ``‘sgrac.uvf'`` and inspect the data again. You should note it is from a different date and the antenna positions are different. The uv-coverage is also different.
This data is already calibrated, but when you inspect the data by `plotms`-ing the amplitude versus uvdist, you may notice something is wrong. How will this affect your map?

You can proceed with the normal clean and see how it is affecting the data. You can flag the data in `plotms`. Don’t spend too much time on imaging the C array data, as we want next to combine the B and C array data. To do this you must use `‘split’` to get the calibrated data on the Galactic centre from the B array data and then `‘concat’` both sets together. Then you can use this combined dataset to image the Galactic centre.

The NRAO image gallery has quite a few radio images of the Galactic centre on a range of scales (http://images.nrao.edu/Galactic_Sources/Galactic_Center). Please note that these were not produced in two class hours from 7 MB of data!